# preprocessing

In [1]:
import pandas as pd

headers = open('data/headers', 'r').read().split('\n')
headers.append('label')
raw = pd.read_csv('data/kddcup.data_10_percent.csv', names=headers, index_col=None)
raw.head()



,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


# Cluster

In [2]:
def preprocess(df: pd.DataFrame, mode='classification', sample=None):
    # pd.set_option('mode.chained_assignment', None)
    df2 = df.query(
        'label == "back." or label == "land." or label == "neptune." or label == "pod." '
        'or label == "smurf." or label == "teardrop." or label == "normal."', inplace=False)

    print(df2.head())

    df['result'] = df['label'].apply(lambda x: 0 if x == 'normal.' else 1)

    df = df[['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
             'land', 'wrong_fragment', 'urgent', 'count', 'serror_rate', 'rerror_rate', 'same_srv_rate',
             'diff_srv_rate',
             'srv_count', 'srv_serror_rate', 'srv_rerror_rate', 'srv_diff_host_rate',
             # 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
             # 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
             # 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
             'label', 'result']]

    df = pd.get_dummies(df, columns=['protocol_type', 'service', 'flag'])

    if sample is not None:
        df = df.sample(n=sample, random_state=1)

    if mode == 'classification':
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        df = df.drop('label', axis=1)
        labels_true = le.fit_transform(df['result'])
        df = df.drop('result', axis=1)
        return (df, labels_true) # X, y
    elif mode == 'cluster':
        labels_true = df['result']
        df = df.drop('label', axis=1)
        df = df.drop('result', axis=1)
        return (df, labels_true) # X, y

df, y = preprocess(raw, mode='cluster', sample=10000)

   duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        181       5450     0   
1         0           tcp    http   SF        239        486     0   
2         0           tcp    http   SF        235       1337     0   
3         0           tcp    http   SF        219       1337     0   
4         0           tcp    http   SF        217       2032     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                   9   
1               0       0    0  ...                  19   
2               0       0    0  ...                  29   
3               0       0    0  ...                  39   
4               0       0    0  ...                  49   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                     1.0                     0.0   
1                     1.0                     0.0   
2                     1.0                     0.0   
3           

In [3]:
import pandas as pd
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt


In [4]:

# 将数据转换为浮点数
df = df.values.astype(np.float32)

# # 数据预处理
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df)

In [5]:
from keras.layers import Input, Dense
from keras.models import Model

# 构建 Autoencoder
input_dim = df.shape[1]
print(input_dim)
encoding_dim = 16  # 您选择的编码维度

input_layer = Input(shape=(input_dim,))
# 编码器层
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# 解码器层
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

encoder = Model(input_layer, encoded)
autoencoder = Model(input_layer, decoded)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')


2023-12-17 19:47:49.167971: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 19:47:49.205824: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 19:47:49.206482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-17 19:47:49.971195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


95


In [6]:
autoencoder.fit(data_scaled, data_scaled, epochs=30, batch_size=128, shuffle=True)

Epoch 1/30
79/79 [==============================] - 1s 2ms/step - loss: 0.9071
Epoch 2/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7596
Epoch 3/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7448
Epoch 4/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7414
Epoch 5/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7399
Epoch 6/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7394
Epoch 7/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7392
Epoch 8/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7390
Epoch 9/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7389
Epoch 10/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7387
Epoch 11/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7385
Epoch 12/30
79/79 [==============================] - 0s 2ms/step - loss: 0.7384
Epoch 13/30
79/79 [==============================

In [7]:
encoded_data = encoder.predict(df)

313/313 [==============================] - 0s 1ms/step


In [8]:
# 应用 BIRCH 聚类
birch_model = Birch(n_clusters = 2)  # 设置聚类数
labels_pred = birch_model.fit_predict(df)

# 评估聚类效果
silhouette = silhouette_score(encoded_data, labels_pred)
ari = adjusted_rand_score(y, labels_pred)
print("Silhouette Coefficient Index:", silhouette)
print("Adjusted Rand index:", ari)

# # 可视化（可选）
# # 这需要您的数据是二维的，或者使用降维方法将其转换为二维
# plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=labels_pred, cmap='viridis')
# plt.title('BIRCH Clustering')
# plt.show()

Silhouette Coefficient Index: 0.9990404
Adjusted Rand index: -0.00015049566622998476


# Classification

In [9]:
# from preprocessing import preprocess
df, y = preprocess(raw, mode='cluster')
result = {}

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

   duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        181       5450     0   
1         0           tcp    http   SF        239        486     0   
2         0           tcp    http   SF        235       1337     0   
3         0           tcp    http   SF        219       1337     0   
4         0           tcp    http   SF        217       2032     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                     1.0   
1               0       0    0  ...                     1.0   
2               0       0    0  ...                     1.0   
3               0       0    0  ...                     1.0   
4               0       0    0  ...                     1.0   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                     0.0                         0.11   
1                     0.0                         0.05   
2                     0.0 

In [10]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))
# from visual import draw
# result['MLP'] = draw(lr, X_test, y_test)


              precision    recall  f1-score   support

           0       0.93      0.98      0.95     19471
           1       0.99      0.98      0.99     79334

    accuracy                           0.98     98805
   macro avg       0.96      0.98      0.97     98805
weighted avg       0.98      0.98      0.98     98805



/home/mininet/Western/cs9636/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
#MLP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report



clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# from visual import draw
# result['MLP'] = draw(clf, X_test, y_test)


              precision    recall  f1-score   support

           0       0.93      0.99      0.96     19471
           1       1.00      0.98      0.99     79334

    accuracy                           0.98     98805
   macro avg       0.96      0.99      0.98     98805
weighted avg       0.98      0.98      0.98     98805



In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=102, max_depth=2, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))
# draw(rf, X_test, y_test)


              precision    recall  f1-score   support

           0       0.97      0.98      0.98     19471
           1       1.00      0.99      0.99     79334

    accuracy                           0.99     98805
   macro avg       0.98      0.99      0.98     98805
weighted avg       0.99      0.99      0.99     98805



In [16]:
import pickle
with open("rf.pickle", "wb") as f:
    pickle.dump(rf, f)